In [1]:
import numpy as np
import libpysal
import spreg
import geopandas as gpd
import polars as pl
from local_functions import *

In [2]:
territorial_information = gpd.read_file("data/transformed/territorial_information_summary.geojson")

In [3]:
for c in territorial_information.columns:
    print(c)

Teritoriālā vienība
Platība (km^2)
Gads
Iedzīvotāju skaits gada sākumā
Gada vidējais iedzīvotāju skaits
Iedzīvotāju dabiskais pieaugums
Migrācijas saldo
Iedzīvotāju skaita izmaiņas - pavisam
Mēneša darba samaksas mediāna
Iedzīvotāji darbspējas vecumā
Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Vispārējie valdības dienesti (Kapitālie izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Kapitālie izdevumi)
Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi)
Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Izgl

In [55]:
y_col = ['Iekšzemes kopprodukts uz vienu iedzīvotāju']
x_col = [
    'Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Sociālā aizsardzība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Iedzīvotāju īpatsvars ar augstāko izglītību',
    'Iedzīvotāji darbspējas vecumā (%)'
]

columns_to_use = [
    'Teritoriālā vienība',
    'Gads',
    'geometry'
] + x_col + y_col

In [56]:
(
    pl.from_dataframe(territorial_information.drop("geometry", axis=1))
    .unpivot(
        on=y_col + x_col,
        index=['Teritoriālā vienība', 'Gads']
    )
    .filter(~pl.col("value").is_null())
    .group_by("variable")
    .agg(
        pl.col("Gads").min().alias("Min year"),
        pl.col("Gads").max().alias("Max year")
    )
    #.to_pandas()
    .group_by(1)
    .agg(
        pl.col("Min year").max(),
        pl.col("Max year").min()
    )
)

literal,Min year,Max year
i32,i32,i32
1,2018,2021


In [57]:
df = territorial_information[columns_to_use]
df = df[df.Gads.isin([2018, 2019, 2020, 2021])]
df = df.fillna(0)
df = df.sort_values(by=['Teritoriālā vienība', 'Gads'])

In [58]:
t = df[df.Gads==2021]
w = libpysal.weights.Queen.from_dataframe(t, ids="Teritoriālā vienība")
w.transform = 'r'

In [59]:
df[x_col]

,Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi) (uz 1 iedz.),"Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi) (uz 1 iedz.)",Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Sociālā aizsardzība (Uzturēšanas izdevumi) (uz 1 iedz.),Iedzīvotāju īpatsvars ar augstāko izglītību,Iedzīvotāji darbspējas vecumā (%)
194,34.295211,0.648652,0.000000,31.477100,31.735802,183.174954,30.986326,20.0,0.606647
237,35.193598,0.479972,0.000000,47.100389,32.658378,191.438786,27.668850,20.4,0.615730
280,34.715664,0.323563,0.336134,35.188336,32.713613,190.928672,29.678151,20.8,0.611899
323,149.512650,8.454388,46.911465,163.783601,108.872885,650.161474,179.703851,21.3,0.621378
172,131.846485,6.924096,60.096126,100.279497,122.442030,527.581608,115.285783,20.3,0.607228
...,...,...,...,...,...,...,...,...,...
332,237.706255,39.040835,27.254330,271.527113,82.906454,650.094918,65.634523,45.5,0.622409
209,267.185816,38.318134,34.437967,60.570814,75.732014,419.022500,123.443081,37.7,0.605057
252,209.356739,44.172188,52.884255,80.233900,75.065087,437.630071,128.074730,38.1,0.614299
295,277.546704,51.206791,63.841547,80.212652,76.662546,440.624078,134.932226,39.1,0.612120


In [60]:
Y_df = df[y_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[y_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [61]:
X_df = df[x_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[x_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [62]:
Y = Y_df.to_numpy()
X = X_df.to_numpy()

fe_lag = spreg.Panel_FE_Lag(Y, X, w, name_y = y_col, name_x=x_col, name_ds="df")

In [63]:
print(fe_lag.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG PANEL - FIXED EFFECTS
-----------------------------------------------------------------------
Data set            :          df
Weights matrix      :     unknown
Dependent Variable  :Iekšzemes kopprodukts uz vienu iedzīvotāju                Number of Observations:         172
Mean dependent var  :      0.0000                Number of Variables   :          10
S.D. dependent var  :   1188.8075                Degrees of Freedom    :         162
Pseudo R-squared    :      0.4927
Spatial Pseudo R-squared:  0.4522
Sigma-square ML     :  713198.482                Log likelihood        :   -1847.641
S.E of regression   :     844.511                Akaike info criterion :    3715.282
                                                 Schwarz criterion     :    3746.757

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic    